l'idée est de réussir à maitriser le lancement et l'utilisation d'un llm en local pour bine comprendre les concepts

l'utilisation etant purement perso dans un premier temps, réussir à lancer et utiliser mon modèle en local serait nickel

In [1]:
%pip install transformers accelerate bitsandbytes huggingface_hub
%pip install torch torchvision torchaudio

  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached huggingface_hub-0.29.3-py3-none-any.whl.metadata (13 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.6.0-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached nvidia_cuda_nvrtc_

In [ ]:
# Importation des bibliothèques
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Connexion à HuggingFace (facultatif pour certains modèles)
# login()

# Téléchargement et chargement du modèle
model_id = "mistralai/Mistral-7B-Instruct-v0.3"  # Ou une autre version de Mistral

# Chargement du tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Chargement du modèle avec quantification pour réduire l'empreinte mémoire
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",          # Utilise CUDA si disponible
    torch_dtype=torch.float16,  # Utilise la précision réduite
    load_in_4bit=True,          # Quantification 4-bit
)

In [ ]:
def generate_response(prompt, max_new_tokens=512):
    # Formatage pour Mistral (format d'instruction)
    formatted_prompt = f"<s>[INST] {prompt} [/INST]"
    
    # Tokenisation du prompt
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    
    # Génération de la réponse
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=max_new_tokens,
        temperature=0.7,        # Contrôle la créativité (plus bas = plus déterministe)
        top_p=0.9,              # Nucleus sampling
        do_sample=True,         # Échantillonnage plutôt que beam search
        repetition_penalty=1.2  # Pénalise la répétition
    )
    
    # Décodage de la réponse
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extraction de la réponse (après l'instruction)
    response = response.split("[/INST]")[-1].strip()
    
    return response

In [ ]:
# Exemple d'utilisation
question = "Explique-moi comment fonctionne un modèle de langage comme Mistral?"
reponse = generate_response(question)
print(reponse)

In [ ]:
# Installation de LangChain
%pip install langchain langchain-community

# Importation des bibliothèques
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from transformers import pipeline

# Création d'un pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.2,
    do_sample=True
)

# Création du modèle LangChain
llm = HuggingFacePipeline(pipeline=pipe)

# Création d'un template de prompt
template = """
[INST] 
{question}
[/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["question"])

# Création de la chaîne
chain = LLMChain(llm=llm, prompt=prompt)

# Utilisation
response = chain.run(question="Explique-moi comment fonctionne un modèle de langage comme Mistral?")
print(response)